In [ ]:
!pip freeze | grep mne

In [ ]:
%load_ext autoreload
%autoreload 2

from utils_postprocess import printDict

import globvars as gv
import mne
import matplotlib.pyplot as plt
from os.path import join as pjoin


defdgen = lambda sz :np.random.uniform(low=-1,high=1, size=sz)

import utils_tSNE as utsne
from utils import runFilePart

sfreq = 256
sfreq_hires = 1024

#from IPython import get_ipython; ipython = get_ipython()
#ipython.magic('run -i ../test_data/gen_test_data6.py')
runFilePart('../test_data/gen_test_data7.py','')

print(test_data_descr)

#rawnames = ['S01_off_move','S01_off_hold','S01_on_move']

# prep_test_data

# _gt means ground truth here
dat_pri_gt           = dat_pri
dat_LFP_hires_pri_gt = dat_LFP_hires_pri

dati = dati
plt.plot(times,dat_pri[dati].T)
plt.figure()
plt.plot(times_hires,
         dat_LFP_hires_pri[dati].T, 
         label=sfo_LFP)
plt.legend()

dat_EMG = defdgen( (2, nbins ) )

# reversed if needed
from utils import makeSimpleRaw
from featlist import selFeatsRegexInds
raws_permod_both_sides2 = {}
for rawi,rawn in enumerate(rawnames):
    raws_permod_both_sides2[rawn] = {}
    chis_LFP = utsne.selFeatsRegexInds(sfo_pri[rawi], 'LFP.*' )
    dat_LFP = dat_pri_gt[rawi][chis_LFP]
    raws_permod_both_sides2[rawn]['LFP'] = makeSimpleRaw(dat_LFP, np.array(sfo_pri[rawi])[chis_LFP], rescale=0 )

    chis_msrc = utsne.selFeatsRegexInds(sfo_pri[rawi], 'msrc.*' )
    dat_msrc = dat_pri_gt[rawi][chis_msrc]
    raws_permod_both_sides2[rawn]['src'] = makeSimpleRaw(dat_msrc, 
        np.array(sfo_pri[rawi])[chis_msrc] , rescale=0)

    raws_permod_both_sides2[rawn]['LFP_hires'] = makeSimpleRaw(dat_LFP_hires_pri_gt[rawi],
        sfo_lfp_hires_pri[rawi], sfreq=sfreq_hires, rescale=0 )
    
    raws_permod_both_sides2[rawn]['EMG'] = makeSimpleRaw(dat_EMG, 
        gv.EMG_per_hand[main_side], sfreq=sfreq, rescale=0)

rawnames_str = ','.join(rawnames)

####################################
import globvars as gv
gv.DEBUG_MODE=1
from IPython import get_ipython; ipython = get_ipython()
#Once that's done, run a magic command like this:
#%debug
raws_permod_both_sides = raws_permod_both_sides2

mstr = 'run -i ../run/run_prep_dat.py -r ' +   rawnames_str + \
        ' --param_file prep_dat_defparams.ini --calc_stats_multi_band 1' +\
        ' --save_dat 1 --save_stats 1 --use_preloaded_raws 1 --allow_CUDA 0'
ipython.magic(mstr)

# %run -i ../run/run_prep_dat.py -r S99_off_move,S99_off_hold,S99_on_move \
#         --param_file prep_dat_defparams.ini --calc_stats_multi_band 1 --save_dat 1 \
#         --use_preloaded_raws 1 --save_stats 1


from IPython import get_ipython; ipython = get_ipython()
#debug

# import warnings
# warnings.filterwarnings("error")
# # with warnings.catch_warnings():
# #     warnings.simplefilter('error')00

# try:

# except ComplexWarning as e:
#     pass

rs = ('%run -i ../run/run_genfeats.py '
' --sources_type parcel_aal --bands crude'
' --src_grouping_fn 10'
' --src_grouping 0 '
' --raw {} '
' --feat_types con,H_act,H_mob,H_compl,rbcorr,bpcorr'
' --Kalman_smooth 0'
' --load_TFR 0'
' --load_CSD 0'
' --save_TFR 0'
' --save_CSD 0'
' --save_bpcorr 0'
' --save_rbcorr 0'
' --load_rbcorr 0'
' --use_existing_TFR 0'
' --use_preloaded_data 0'
' --allow_CUDA 0'
' --load_only 0'
' --show_plots 0'
' --plot_types ,'
' --prescale_data 1'
' --exit_after {}'
' --normalize_TFR separate'
' --scale_data_combine_type no'
' --baseline_int_type entire'
' --n_jobs 1'
' --save_feat 1'
' --feat_stats_artif_handling reject'
' --scale_data_combine_type no '
' --rbcorr_use_local_means 0'
' --output_subdir test'
' --stats_fn_prefix stats_{}_{}_ '
' --param_file genfeats_defparams.ini ')

exit_after = 'end'
#exit_after = 'prescale_data'
#exit_after = 'load'
mstr2 = rs.format(rawnames_str,exit_after,rawnames_str[:3],len(rawnames))

#%debug

ipython.magic(mstr2)

#np.savez(pjoin(gv.data_dir,'test.npz') ,globals() )

In [ ]:
print( rawnames )

In [ ]:
group_labels_dict

In [ ]:
rawnames = ['S95_off_hold', 'S95_off_move']
import globvars as gv

In [ ]:
#rawnames_str = 'S95_off_move'
#rawnames_str = 'S95_off_hold,S95_off_move'
rawnames_str = ','.join(rawnames)
gv.DEBUG_MODE=1

%load_ext autoreload
%autoreload 2

#    --calc_VIF 1\
#    --feat_types Hjorth,con,rbcorr,bpcorr\
#--parcel_group_names motor-related\
#--EBM_featsel_feats all,heavy,VIFsel
from IPython import get_ipython; ipython = get_ipython()
mstr_ML = '''%run -t -i ../run/run_ML.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
    --src_grouping 0 \
    --prefix test \
    --skip_XGB 0\
    --mods LFP,msrc\
    --LFP_related_only 1\
    --feat_types con\
    --allow_CUDA 0\
    --subskip_fit 1 \
    --n_splits 4 \
    --calc_MI 0 \
    --save_output 1\
    --skip_LDA 0\
    --calc_VIF 0\
    --compute_ICA  0\
    --use_ICA_for_classif 0\
    --parcel_types !Frontal_Sup\
    --heavy_fit_red_featset 0\ 
    --featsel_only_best_LFP 0\
    --calc_Boruta 0\
    --groupings_to_use merge_nothing --int_types_to_use basic\
    --input_subdir test --output_subdir test\
    --raw {} --load_only 0 --show_plots 1\
    --XGB_tune_param 1\
    --XGB_tree_method exact\
    --param_file ML_defparams.ini \
    --featsel_method XGB_Shapley,interpret_DPEBM,interpret_EBM\
    --EBM_featsel_feats all
    --EBM_compute_pairwise 0\
    --label_groups_to_use subj_medcond_task \
    --exit_after {}\
    --prep_for_clf_only 0 \
    --EBM_CV 1 \
    --EBM_tune_param 1 \
    --EBM_tune_max_evals 2 \
    --EBM_balancing weighting
    --load_EBM_params_auto 0'''

#--skip_LDA 1\
#merge_all_not_trem
#%debug

exit_after = 'end'
#exit_after = 'XGB_main'
#exit_after = 'XGB_search_LFP'
mstr_ML = mstr_ML.format(rawnames_str,exit_after)


#%debug
ipython.magic(mstr_ML)

In [ ]:
!ls /home/demitau/data/test/S95/test_/basic/merge_nothing/featsel_per_method/interpret_EBM/all/

In [ ]:
d = {'binning': 'uniform', 'learning_rate': 0.005, 'max_bins': 64, 'max_interaction_bins': 32, 'max_leaves': 5, 'min_samples_leaf': 4, 'outer_bags': 12, 'validation_size': 0.1}

In [ ]:
results_cur['featsel_per_method']['interpret_EBM']['all'].keys()

In [ ]:
dict(results_cur['cmd'][0])

In [ ]:
resc_['featsel_per_method'][fsh][featsel_feat_subset_name].keys()

In [ ]:
resc_['featsel_per_method'][fsh][featsel_feat_subset_name]['perf_per_cp']

In [ ]:
#ebm_creopts_tuned

In [ ]:
EBM_tune_param and load_EBM_params_auto \
                                and os.path.exists(fname_ML_full_intermed_light)

In [ ]:
results_cur['XGB_analysis_versions']['all_present_features']['perf_per_cp']

In [ ]:
results_cur['XGB_analysis_versions']['all_present_features']['perf_dict'].keys()

In [ ]:
n_splits

In [ ]:
r0 = utsne.getPredPowersCV(clf_XGB,X_cur,y_cur,
    class_ind_to_check_lenc, printLog = 0,
    n_splits=n_splits, add_fitopts=add_fitopts,
    add_clf_creopts=add_clf_creopts,
    ret_clf_obj=True, seed=0)

from utils_tSNE import extractSubperfs
perf_per_cp = extractSubperfs(X_cur,class_labels_good_for_classif, class_labels_good_for_classif_nm, 
                revdict_lenc, revdict_lenc_nm,
               class_ind_to_check_lenc, r0['clf_objs' ] , r0['test_indices_list'] )

In [ ]:
r0['test_indices_list']

In [ ]:
revdict

In [ ]:
revdict_nm

In [ ]:
lab_enc.inverse_transform([0])

In [ ]:
r0['clf_objs']

In [ ]:
'a' in ['a', 'b']

In [ ]:
if 'a' in {'a':5}:
    print('f')

In [ ]:
#ivalis_tb_indarrays_merged

In [ ]:
class_labels

In [ ]:
a,b = zip(*class_ids_grouped.items())  
dict( zip(b,a))

In [ ]:
discard_remaining_int_types_during_fit

In [ ]:
numpoints_per_class_id

In [ ]:
featsel_per_method['interpret_EBM']['all']['perf_per_cp']

In [ ]:
featsel_per_method['interpret_EBM']['all']['perf_dict']

In [ ]:
featsel_per_method['interpret_DPEBM']['all']['perf_per_cp']

In [ ]:
featsel_per_method['interpret_DPEBM']['all'].keys()

In [ ]:
featsel_per_method['interpret_DPEBM']['all']['perf_per_cp']

In [ ]:
featsel_per_method['interpret_DPEBM']['all']['perf_dict']#['clf_objs']

In [ ]:
pp.printSizeInfo(r)

In [ ]:
r['featsel_per_method']['interpret_DPEBM']['all'].keys()

In [ ]:
#%debug
r = pp.removeLargeItems(results_cur)
r['featsel_per_method']['interpret_DPEBM']['all']['ebmobj']

In [ ]:
r['featsel_per_method']['interpret_DPEBM']['all']['perf']

In [ ]:
#%debug
info_cur = utsne.computeEBM(X_EBM,y_EBM,EBM,ebm_creopts,revdict_lenc,
        class_ind_to_check_lenc, class_labels_good_nm, revdict_nm,
                            n_splits=n_splits,
        EBM_compute_pairwise=EBM_compute_pairwise,
        EBM_CV=EBM_CV, featnames_ebm=featnames_ebm,
        tune_params = 0, params_space=params_space,
                            max_evals=EBM_tune_max_evals)


In [ ]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')


In [ ]:
X_,y_ = oversample.fit_resample(X,y)

In [ ]:
len(y)

In [ ]:
len(oversample.sample_indices_)

In [ ]:
help(RandomOverSampler)

In [ ]:
help(oversample.fit_resample)

In [ ]:
info_cur['perf_per_cp']

In [ ]:
len( info_cur['perf_per_cp'] )

In [ ]:
np.exp(xx)

In [ ]:
xx=0.021991313767351812

In [ ]:
1 / (1 + np.exp(xx))

In [ ]:
ebm, expl_data= info_cur

In [ ]:
expl_data

In [ ]:
sum(y)  / len(y)

In [ ]:
sum( ebm.predict(X) ) / sum(y)

In [ ]:
ebm.explain_local(X).__dict__.keys()

In [ ]:
len(X)

In [ ]:
len( ebm.explain_local(X)._internal_obj['specific'] )

In [ ]:
help(ebm.explain_local)

In [ ]:
ebm.explain_local(X,y)._internal_obj['specific'][0]

In [ ]:
ebm.predict_proba(X)

In [ ]:
help(ebm.predict_proba)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
#X_orig,y_orig = X,y
X_,y_ = oversample.fit_resample(X,y)
    

In [ ]:
numpoints_per_class_id

In [ ]:
type(y_)

In [ ]:
utsne.countClassLabels(y_)

In [ ]:
from hyperopt import hp,fmin,tpe, Trials, STATUS_OK

space = { 'max_bins': hp.choice('max_bins',[64,128,256,384, 512]),
 'outer_bags':hp.choice('outer_bags',[2,4,8,12,16]),
 'learning_rate':hp.choice('learning_rate',[1e-3,5e-3,1e-2,2e-2]),
 'validation_size':hp.choice('validation_size',[0.10,0.15, 0.2, 0.3]),
 'min_samples_leaf':hp.choice('min_samples_leaf',[2,3,4,5,7]),
 'max_leaves':hp.choice('max_leaves',[2,3,5,7]),
 'binning': hp.choice('binning',['quantile','quantile_humanized', 'uniform'])
}
if len(numpoints_per_class_id) > 2:
    nfeats = len(featnames_ebm)
    interaction_d = {'interactions': hp.quniform('interactions',0, (nfeats*(nfeats-1) // 2),1),
    'max_interaction_bins':hp.choice('max_interaction_bins', [16,32,64])
    }
    space.update(interaction_d)

space2 = {  'max_bins': hp.choice('max_bins',[16,32,64,128,256]),
 #quantile_humanized
 'outer_bags':hp.choice('outer_bags',[1,2,4,8]),
 'learning_rate':hp.choice('learning_rate',[1e-3,5e-3,1e-2,2e-2]),
 #'validation_size':hp.choice('validation_size',[0., 0.10, 0.2, 0.3]),
 'min_samples_leaf':hp.choice('min_samples_leaf',[2,3,4,5,7]),
 'max_leaves':hp.choice('max_leaves',[2,3,5,7]),
 'bin_budget_frac':hp.choice( 'bin_budget_frac', [0.05, 0.1, 0.2] ) ,
 'epsilon': hp.choice('epsilon', [0.75, 1, 1.25]),
 'delta': hp.choice('delta', [1e-6, 1e-5, 2e-5, 1e-4, 1e-3])
}


#space2 = {  'max_bins': hp.choice('max_bins',[16,32,64,128,256]) }
#composition = 'classic' or 'gdp'
#'binning': hp.choice('binning',['quantile','quantile_humanized', 'uniform'])
#privacy_schema: User specified min/maxes for numeric features as dictionary. Only applicable when binning is "private".

# for feati in range(X.shape[1]):
#     np.quantile(X,)
q = 1e-1
qX = np.quantile(X_for_heavy,[q,1-q], axis=0) # 2 x X.shape[0]
privacy_schema = dict( zip( range(X.shape[0]), list( zip(*qX) )  ) )
privacy_schema['target'] = (0,0)
ebm_creopts['privacy_schema'] = privacy_schema # ONLY FOR DBEx

#         outer_bags=1,
#         # Boosting
#         learning_rate=0.01,
#         validation_size=0,
#         max_rounds=300,
#         max_bins=32,
#         binning="private",
#         epsilon=1,
#         delta=1e-5,
#         composition='gdp',
#         bin_budget_frac=0.1,
#         privacy_schema=None,

# space = {'name':hp.choice('param_name',['val1','val2']),
#         'pname2':hp.uniform('pname2', 0, 0.5)}
#hp.quniform(label, low, high, q)

from sklearn.model_selection import cross_val_score
X_train = X_for_heavy [::2]
y_train = class_labels_good_for_classif [::2]

In [ ]:
def objective(space_loc):
    import time
    st_time =time.time()
    ebm_creopts_loc = dict(ebm_creopts.items())
    ebm_creopts_loc.update(space_loc)
    ebm_creopts_loc['n_jobs'] = 1
    ebm_creopts_loc['composition'] = 'gdp'
    #model = EBM(**space_loc)
    model = EBM(**ebm_creopts_loc)
    accuracy = cross_val_score(model,X_train,y_train,cv=4, scoring='balanced_accuracy').mean()
    
    print(space_loc)
    return {'loss':-accuracy, 'status':STATUS_OK, 'creopts':ebm_creopts_loc, 'effparams':space_loc,
           'time':time.time() - st_time}

#%debug
trials = Trials()
max_evals = 4
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
best

# trials.trials - a list of dictionaries representing everything about the search
# trials.results - a list of dictionaries returned by 'objective' during the search
# trials.losses() - a list of losses (float for each 'ok' trial)
# trials.statuses() - a list of status strings

In [ ]:
trials.results[ np.argmin( trials.losses() ) ]['creopts']

In [ ]:
trials.results

In [ ]:
for tr in trials:
    print( tr['result']['effparams'])

In [ ]:
list( trials)[0]

In [ ]:
list( trials)[0]['creopts']

In [ ]:
help(EBM)

In [ ]:
EBM

In [ ]:
from interpret.privacy import DPExplainableBoostingClassifier as EBM


In [ ]:
from interpret.glassbox import DPExplainableBoostingClassifier as EBM


In [ ]:
help( interpret.glassbox.ebm)

In [ ]:
EBM_compute_pairwise

In [ ]:
set(y)

In [ ]:
                            EBM_CV = 1
                            info_cur = utsne.computeEBM(X,y,EBM,ebm_creopts,revdict,
                                        class_ind_to_check_lenc, n_splits=n_splits,
                                        EBM_compute_pairwise=EBM_compute_pairwise,
                                        EBM_CV=EBM_CV, featnames_ebm=featnames_ebm)
                            info_cur['feature_indices_used'] = featis
                            featsel_info[featsel_feat_subset_name] = info_cur


In [ ]:
#rawnames_str = 'S95_off_move'
#rawnames_str = 'S95_off_hold,S95_off_move'
rawnames_str = ','.join(rawnames)
gv.DEBUG_MODE=1

%load_ext autoreload
%autoreload 2

#    --calc_VIF 1\
#    --feat_types Hjorth,con,rbcorr,bpcorr\
#--parcel_group_names motor-related\
from IPython import get_ipython; ipython = get_ipython()
mstr_ML = '''%run -t -i ../run/run_ML.py --sources_type parcel_aal --bands_type crude --src_grouping_fn 10\
    --src_grouping 0 \
    --prefix test \
    --skip_XGB 0\
    --mods LFP,msrc\
    --feat_types H_act,H_mob,H_compl\
    --allow_CUDA 0\
    --subskip_fit 1 \
    --n_splits 4 \
    --calc_MI 0 \
    --save_output 1\
    --skip_LDA 1\
    --calc_VIF 0\
    --compute_ICA  0\
    --use_ICA_for_classif 0\
    --parcel_types !Frontal_Sup\
    --heavy_fit_red_featset 0\ 
    --featsel_only_best_LFP 0\
    --calc_Boruta 0\
    --groupings_to_use merge_movements --int_types_to_use basic\
    --input_subdir test --output_subdir test\
    --raw {} --load_only 0 --show_plots 1\
    --XGB_tune_param 1\
    --XGB_tree_method exact\
    --param_file ML_defparams.ini \
    --featsel_method XGB_Shapley,interpret_EBM\
    --EBM_featsel_feats all,heavy,VIFsel
    --EBM_compute_pairwise 0\
    --label_groups_to_use subj_medcond_task
    --exit_after {}\
    --prep_for_clf_only 0'''

#--skip_LDA 1\

#%debug

exit_after = 'end'
#exit_after = 'load'
#exit_after = 'XGB_main'
#exit_after = 'XGB_search_LFP'
mstr_ML = mstr_ML.format(rawnames_str,exit_after)


#%debug
ipython.magic(mstr_ML)

In [ ]:
                        resc_ = fe['results_light'][()]
                        rc = resc_['XGB_analysis_versions']['all_present_features']
                        add_clf_creopts_tuned  = rc.get('add_clf_creopts',None)
                        add_fitopts_tuned      = rc.get('add_fitopts',None)
                        print('-------- Loaded XGB parameters from file!')


In [ ]:
results_cur.keys()

In [ ]:
expl_loc = ebm.explain_local(X[:3],y[:3])

In [ ]:
expl_loc.data()

In [ ]:
load_XGB_params_auto

In [ ]:
fname_ML_full_intermed_light

In [ ]:
load_XGB_params_auto and os.path.exists(fname_ML_full_intermed_light)


In [ ]:
help(ebm.fit)

In [ ]:
feature_names_all

In [ ]:
feature_names_all2 = list(feature_names_all) + ['H_act_msrcR_9_5_c0','H_mob_msrcR_9_5_c0']

In [ ]:
features_to_use = ['H_act']

In [ ]:
    selected_feat_inds =  filterFeats(feature_names_all2, chnames_LFP,
                                      LFP_related_only, parcel_types,
                                      remove_crossLFP, cross_couplings_only,
                                      self_couplings_only, fbands_to_use,
                                      features_to_use, fbands_per_mod, feat_types_all,
                                      data_modalities, data_modalities_all,
                                      msrc_inds, parcel_group_names,
                                      roi_labels,srcgrouping_names_sorted,
                                      src_file_grouping_ind, fbands_def,
                                      fband_names_fine_inc_HFO, use_lfp_HFO,
                                      use_main_LFP_chan, mainLFPchan,
                                      mainLFPchan_new_name_templ,
                                      printLog=1)


In [ ]:
np.array(feature_names_all2)[selected_feat_inds]

In [ ]:
featnames

In [ ]:
X_for_heavy.shape

In [ ]:
featnames

In [ ]:
expl = utsne.shapr_proxy(X_for_heavy,class_labels_for_heavy,featnames)

In [ ]:
expl[4]

In [ ]:
pca.components_.shape

In [ ]:
pca

In [ ]:
tt = np.random.uniform(-1,1,size=(1000,10))
tt[:,:-2] = 0
pca = PCA(n_components=0.6)
pca.fit(tt)
ica = FastICA(n_components=pca.components_.shape[0],
                random_state=0, max_iter=max_iter)
ica.fit(  pca.transform(tt))


In [ ]:
pca.n_components_

In [ ]:
from utils import pcaica2featCoef
mix_appl = pcaica2featCoef(pca,ica)


In [ ]:
mix_appl[:,0], mix_appl[:,1]

In [ ]:
mix_appl.shape

In [ ]:
pca.components_.shape

In [ ]:
mix_appl[:,0]

In [ ]:
pca.__dict__

In [ ]:
ica.__dict__

In [ ]:
X_for_heavy.shape

In [ ]:
pca_derived_featinds

In [ ]:
results_cur['XGB_analysis_versions']['all_present_features']['across']

In [ ]:
results_cur['featsel_per_method']['interpret_EBM']

# Plot class labels

In [ ]:
Xtimes_concat = wbd_merged[0] / sfreq
markersize = 4
# TODO: concat xtimes, show for several datasets joined, with EMG
if len(artif_naive_bininds):
    mask =  [-2]*len(artif_naive_bininds)
    #plt.plot(Xtimes_pri[0][bininds_concat_bad], bininds_concat_bad)
    plt.plot(Xtimes_concat[artif_naive_bininds],mask, lw=0, marker='x', 
             markersize=markersize,
            label='naive bad inds')

bininds_concat_bad = \
    np.setdiff1d(np.arange(Xconcat_imputed.shape[0]),bininds_concat_good_yes_label) 
mask =  [-1]*len(bininds_concat_bad)
#plt.plot(Xtimes_pri[0][bininds_concat_bad], bininds_concat_bad)
plt.plot(Xtimes_concat[bininds_concat_bad],mask, lw=0, marker='x', 
         markersize=markersize,
        label='bad inds')

clgu = sorted(set(class_labels_good))
for cidi,class_id in enumerate( clgu):
    orig_inds = bininds_concat_good_yes_label[np.where(class_labels_good==class_id)[0]]
    mask =  [cidi]*len(orig_inds)
    plt.plot(Xtimes_concat[orig_inds],mask, lw=0, marker='o', markersize=markersize, 
             label=class_label_names_ordered[cidi])

for lblid in sorted(set(class_labels_good_for_classif) ):
    binds_cur = np.where(class_labels_good_for_classif == lblid)[0]
    binds_cur = bininds_concat_good_yes_label[binds_cur]
    mask =  [lblid + len(clgu) ]*len(binds_cur)
    plt.plot(Xtimes_concat[binds_cur], mask, lw=0, marker='o', markersize=markersize,
             label = 'forclf_' +class_label_names_ordered[lblid])
    
plt.legend(loc = (1,0))
plt.xlim(0,datlen_s * len(X_pri))


In [ ]:
test_plots_descr

In [ ]:
special_chns

In [ ]:
from plots import plotFeatsWithEverything
#%debug
plotFeatsWithEverything(dat_pri, times_pri, X_pri, 
                            Xtimes_pri, dat_lfp_hires_pri, times_hires_pri,
                            rawnames,
                            subfeature_order_pri, subfeature_order_newsrcgrp_pri,
                            subfeature_order_lfp_hires_pri,
                            anndict_per_intcat_per_rawn,
                            featnames, wbd_H_pri,
                            sfreq, raw_perband_flt_pri, raw_perband_bp_pri,
                            scale_data_combine_type,
                            stats_multiband_flt, stats_multiband_bp,
                            test_plots_descr, special_chns,
                           fband_names_inc_HFO )


In [ ]:
VIF_truncation_6 = VIF_truncation

In [ ]:
VIF_truncation_7 = VIF_truncation

In [ ]:
VIF_truncation_6

In [ ]:
VIF_truncation_7

In [ ]:
info_per_cp.keys()

In [ ]:
#explain_global = info_per_cp[('trem_L', 'notrem_L')]['explainer']
explain_global = info_per_cp[('trem_L', 'hold_L&move_L')]['explainer']

from interpret import show

show(explain_global)

In [ ]:
from utils_tSNE import getScoresPerClass
r = getScoresPerClass(class_labels_good_for_classif,featsel_per_method['XGB_Shapley']['scores'],1)



scores,bias = r

#class_label_names_ordered

tremind = class_label_names_ordered.index('trem_L')
si = np.argmax(scores[tremind] )

bestfeats = []
bestfeats_nice = []
feat_comments = []
for ind,lblstr in enumerate(class_label_names_ordered):
    si = np.argmax(scores[ind] )
    if featsel_on_VIF:
        feat_inds =  VIF_truncation['colinds_good_VIFsel']
    else:
        feat_inds = np.arange(len(featnames))
    tmp = np.array(featnames_nice)[ feat_inds[si] ]
    bestfeats_nice += [tmp]
    tmp = np.array(featnames)[ feat_inds[si] ]
    bestfeats += [tmp]
    feat_comments += ['best for ' + lblstr]

print(bestfeats_nice)
featnames_sel = bestfeats

In [ ]:
print(scores), bias

In [ ]:
feat_comments

In [ ]:
from plots import plotFeatsAndRelDat
#%debug
plotFeatsAndRelDat(rawnames, featnames_sel, dat_pri,subfeature_order_pri,
                X_pri,[featnames]*len(rawnames),times_pri,Xtimes_pri,
                subfeature_order_newsrcgrp_pri, wbd_H_pri,
                dat_hires_pri=dat_lfp_hires_pri,
                chnames_all_hires_pri = subfeature_order_lfp_hires_pri,
                times_hires_pri=times_hires_pri,
                anndict_per_intcat_per_rawn=anndict_per_intcat_per_rawn, sfreq=sfreq,
                  feat_comments=feat_comments)

plt.tight_layout()

In [ ]:
VIF_truncation.keys()

In [ ]:
exogs_list[-1]

# Look at regressable features

In [ ]:
colinds_good_VIFsel,colinds_bad_VIFsel[-1]

In [ ]:
featnames_sel = np.array(featnames)[list(colinds_good_VIFsel) +\
                                    [colinds_bad_VIFsel[-1]] ]
print(featnames_sel)
feat_comments = len(colinds_good_VIFsel) * [''] + ['last_bad']

In [ ]:
from plots import plotFeatsAndRelDat
#%debug
plotFeatsAndRelDat(rawnames, featnames_sel, dat_pri,subfeature_order_pri,
                X_pri,[featnames]*len(rawnames),times_pri,Xtimes_pri,
                subfeature_order_newsrcgrp_pri, wbd_H_pri,
                dat_hires_pri=dat_lfp_hires_pri,
                chnames_all_hires_pri = subfeature_order_lfp_hires_pri,
                times_hires_pri=times_hires_pri,
                anndict_per_intcat_per_rawn=anndict_per_intcat_per_rawn, sfreq=sfreq,
                  feat_comments=feat_comments)

plt.tight_layout()

In [ ]:
linreg_dict

In [ ]:
colinds_good_VIFsel

In [ ]:
inds = list(colinds_good_VIFsel) + [colinds_bad_VIFsel[-1]]
print(inds)

vif_dict,tole,linreg_dict = utsne.sklearn_VIF(Xconcat_to_fit[:,inds], [inds.index(colinds_bad_VIFsel[-1]) ], n_jobs=-1,
                  search_worst=0, return_obj='all')

regressor = list(linreg_dict.values() )[0]
Xsummed = np.sum( Xconcat_to_fit[:,inds[:-1]] * regressor.coef_[None,:], axis=1)
print(regressor.coef_)

plt.plot(Xsummed)
plt.plot(Xconcat_to_fit[:,inds[-1]])

In [ ]:
linreg_dict[5].coef_

In [ ]:
linreg_dict[5].